In [1]:
!pip install Scrapy

In [2]:
!pip install Boto3

   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   ----------- --------------------------- 41.0/139.2 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 139.2/139.2 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   -- ------------------------------------- 0.9/13.3 MB 28.7 MB/s eta 0:00:01
   -------- ------------------------------- 2.8/13.3 MB 29.9 MB/s eta 0:00:01
   ------------- -------------------------- 4.6/13.3 MB 36.5 MB/s eta 0:00:01
   --------------------- ------------------ 7.2/13.3 MB 38.6 MB/s eta 0:00:01
   --------------------------- ------------ 9.1/13.3 MB 41.6 MB/s eta 0:00:01
   ------------------------------ --------- 10.0/13.3 MB 42.7 MB/s eta 0:00:01
   ------------------------------ --------- 10.0/13.3 MB 42.7 MB/s eta 0:00:01
   ------------------------------------- -- 12.6/13.3 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------  13.3/13.3 MB 36.3 MB/s eta 0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.35.92 which is incompatible.


In [3]:
import requests
import time
import pandas as pd
import json
import plotly.express as px

from bs4 import BeautifulSoup

import scrapy
from scrapy.crawler import CrawlerProcess

import logging

import boto3

In [4]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

url_nominatim = "https://nominatim.openstreetmap.org"

In [5]:
df_cities = pd.DataFrame(cities, columns = ["city"])
df_cities.head()

,city
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen


In [6]:
df_cities = pd.DataFrame(cities, columns = ["city"])

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

i = 0

for city in cities:
  payload = {
      "q": city,
      "format": "jsonv2",
      "limit": 1
      }

  response = requests.get(url_nominatim+"/search", headers=headers, params=payload)
  df_cities.loc[i, "lat"] = response.json()[0]["lat"]
  df_cities.loc[i, "lon"] = response.json()[0]["lon"]
  df_cities.loc[i, "place_id"] = response.json()[0]["place_id"]

  i += 1
  time.sleep(1)

df_cities.head()

,city,lat,lon,place_id
0,Mont Saint Michel,48.6359541,-1.511459954959514,263784110.0
1,St Malo,48.649518,-2.0260409,263088376.0
2,Bayeux,49.2764624,-0.7024738,262975271.0
3,Le Havre,49.4938975,0.1079732,89704449.0
4,Rouen,49.4404591,1.0939658,90829492.0


In [8]:
df_cities.to_csv("data/cities_location.csv")

In [9]:
df_cities = pd.read_csv("data/cities_location.csv")
df_cities.head()

,Unnamed: 0,city,lat,lon,place_id
0,0,Mont Saint Michel,48.635954,-1.511460,263784110.0
1,1,St Malo,48.649518,-2.026041,263088376.0
2,2,Bayeux,49.276462,-0.702474,262975271.0
3,3,Le Havre,49.493898,0.107973,89704449.0
4,4,Rouen,49.440459,1.093966,90829492.0


Appel de l'api openweathermap et récupération des infos importantes

In [10]:
api_key = open("data/api_key.txt", "r").readlines()

api_key[0]

df_weather = pd.DataFrame()

def get_weather(df_weather, city, json_object):
  for i in range(1, 8):
    if("rain" in json_object["daily"][i]):
      rain = json_object["daily"][i]["rain"] * json_object["daily"][i]["pop"]
    else:
      rain = 0

    list_tmp = [city, json_object["daily"][i]["dt"],
                json_object["daily"][i]["feels_like"]["day"],
                json_object["daily"][i]["humidity"],
                rain
                ]
    df_tmp = pd.DataFrame([list_tmp], columns=["city", "timestamp", "feels_like", "humidity", "rain"])
    df_weather = pd.concat([df_weather, df_tmp])

  return df_weather

for i in df_cities.index:
  url = "https://api.openweathermap.org/data/3.0/onecall?lat={}&lon={}&APPID={}&exclude={}&units={}".format(
      df_cities.loc[i, "lat"],
      df_cities.loc[i, "lon"],
      api_key[0],
      'hourly',
      "metric")

  response_weather = requests.get(url)

  df_weather = get_weather(df_weather, df_cities.loc[i, "city"], response_weather.json())

In [11]:
display(df_weather)

,city,timestamp,feels_like,humidity,rain
0,Mont Saint Michel,1736164800,4.36,66,6.9600
0,Mont Saint Michel,1736251200,3.02,69,2.8300
0,Mont Saint Michel,1736337600,-2.07,95,21.5100
0,Mont Saint Michel,1736424000,1.76,68,6.6300
0,Mont Saint Michel,1736510400,1.91,80,0.1207
...,...,...,...,...,...
0,La Rochelle,1736337600,11.94,85,12.1000
0,La Rochelle,1736424000,9.89,92,8.5200
0,La Rochelle,1736510400,1.86,79,2.6400
0,La Rochelle,1736596800,0.61,70,0.0000


main => feels_like

weather => main

pop

In [12]:
df_cities_weather = df_cities.merge(df_weather, on="city")

df_cities_weather.reset_index(drop=True, inplace=True)
df_cities_weather.rename(columns={"Unnamed: 0": "id_city"}, inplace=True)

display(df_cities_weather)

,id_city,city,lat,lon,place_id,timestamp,feels_like,humidity,rain
0,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736164800,4.36,66,6.9600
1,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736251200,3.02,69,2.8300
2,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736337600,-2.07,95,21.5100
3,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736424000,1.76,68,6.6300
4,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736510400,1.91,80,0.1207
...,...,...,...,...,...,...,...,...,...
240,34,La Rochelle,46.159732,-1.151595,266833066.0,1736337600,11.94,85,12.1000
241,34,La Rochelle,46.159732,-1.151595,266833066.0,1736424000,9.89,92,8.5200
242,34,La Rochelle,46.159732,-1.151595,266833066.0,1736510400,1.86,79,2.6400
243,34,La Rochelle,46.159732,-1.151595,266833066.0,1736596800,0.61,70,0.0000


In [13]:
df_cities_weather["date"] = pd.to_datetime(df_cities_weather["timestamp"], unit="s").dt.date

df_cities_weather.head(10)

,id_city,city,lat,lon,place_id,timestamp,feels_like,humidity,rain,date
0,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736164800,4.36,66,6.9600,2025-01-06
1,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736251200,3.02,69,2.8300,2025-01-07
2,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736337600,-2.07,95,21.5100,2025-01-08
3,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736424000,1.76,68,6.6300,2025-01-09
4,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736510400,1.91,80,0.1207,2025-01-10
5,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736596800,2.07,74,0.0000,2025-01-11
6,0,Mont Saint Michel,48.635954,-1.511460,263784110.0,1736683200,3.93,69,0.0000,2025-01-12
7,1,St Malo,48.649518,-2.026041,263088376.0,1736164800,4.75,65,6.0800,2025-01-06
8,1,St Malo,48.649518,-2.026041,263088376.0,1736251200,3.62,64,2.4200,2025-01-07
9,1,St Malo,48.649518,-2.026041,263088376.0,1736337600,-1.86,96,23.0900,2025-01-08


In [14]:
fig = px.scatter_mapbox(df_cities_weather, lat="lat", lon="lon", color="feels_like",
                        zoom=4, mapbox_style="carto-positron", animation_frame = "date",
                        hover_name='city'
                        )
fig.write_html('data/feels_like.html')
fig.show()

In [15]:
df_cities_weather["feels_like_mean"] = df_cities_weather["feels_like"].groupby(df_cities_weather["city"]).transform("mean")

fig = px.scatter_mapbox(df_cities_weather, lat="lat", lon="lon", color="feels_like_mean",
                        zoom=4, mapbox_style="carto-positron",
                        hover_name='city'
                        )
fig.write_html('data/feels_like_mean.html')
fig.show()

In [16]:
df_cities_weather["rain_mean"] = df_cities_weather["rain"].groupby(df_cities_weather["city"]).transform("mean")

fig = px.scatter_mapbox(df_cities_weather, lat="lat", lon="lon", color="rain_mean",
                        zoom=4, mapbox_style="carto-positron",
                        hover_name='city'
                        )
fig.write_html('data/rain_mean.html')
fig.show()

In [24]:
df_best_cities = df_cities_weather[["city", "lat", "lon", "place_id", "rain_mean", "feels_like_mean"]].drop_duplicates().sort_values(by=["rain_mean", "feels_like_mean"], ascending=[True, False]).reset_index(drop=True).head(5)
df_best_cities.head(10)

,city,lat,lon,place_id,rain_mean,feels_like_mean
0,Saintes Maries de la mer,43.451592,4.427720,74989696.0,0.010029,7.824286
1,Aigues Mortes,43.566152,4.191540,74835274.0,0.017714,8.348571
2,Nimes,43.837425,4.360069,75374635.0,0.366600,7.335714
3,Collioure,42.525050,3.083155,76480430.0,0.457200,10.118571
4,Aix en Provence,43.529842,5.447474,74605205.0,0.736657,6.970000


In [25]:
id_booking = ['-1465138', '-1406800', '-1455068', '-1421032', "-1406939"]

df_best_cities['id_booking'] = id_booking

In [26]:
df_cities_weather.to_csv("data/cities_weather.csv", index=False)
df_best_cities.to_csv("data/best_cities.csv", index=False)

In [27]:
df_best_cities = pd.read_csv("data/best_cities.csv", )
df_best_cities.head()

,city,lat,lon,place_id,rain_mean,feels_like_mean,id_booking
0,Saintes Maries de la mer,43.451592,4.427720,74989696.0,0.010029,7.824286,-1465138
1,Aigues Mortes,43.566152,4.191540,74835274.0,0.017714,8.348571,-1406800
2,Nimes,43.837425,4.360069,75374635.0,0.366600,7.335714,-1455068
3,Collioure,42.525050,3.083155,76480430.0,0.457200,10.118571,-1421032
4,Aix en Provence,43.529842,5.447474,74605205.0,0.736657,6.970000,-1406939


Get Hotels

In [66]:
!python src/scraping_booking_ok.py

2025-01-05 15:33:48 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-01-05 15:33:48 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.11.0, Python 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-01-05 15:33:48 [scrapy.addons] INFO: Enabled addons:
[]
2025-01-05 15:33:48 [scrapy.extensions.telnet] INFO: Telnet Password: 00e8d3d515e53c58
2025-01-05 15:33:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-01-05 15:33:48 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
           

In [30]:
scrapy.__version__

'2.11.1'

In [31]:
import scrapy
from scrapy.crawler import CrawlerProcess

import logging
from urllib.parse import urlencode
import pandas as pd

class BookingSpider(scrapy.Spider):
  name = "booking"

  custom_settings = {
            "REQUEST_FINGERPRINTER_IMPLEMENTATION": "2.7"
  }

  def start_requests(self):
    df_best_cities = pd.read_csv("best_cities.csv", )

    for index, row in df_best_cities.iterrows():
      url = 'https://www.booking.com/searchresults.fr.html'
      params = {'ss': row.city, 'lang': 'fr', 'dest_id': row.id_booking, 'dest_type': 'city'}
      full_url = f'{url}?{urlencode(params)}'

      yield scrapy.Request(url=full_url, callback=self.parse, cb_kwargs={'city': row.city})

  def parse(self, response, city):
    hotels = response.xpath('//div[@class="d4924c9e74"]/div[@class="c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4"]')

    for hotel in hotels:
      url = hotel.xpath('div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a/@href').get()
      if url:
        yield response.follow(url, callback=self.parse_hotel, cb_kwargs={'city': city, 'url': url})

  def parse_hotel(self, response, city, url):

    try:
      score = response.xpath('//div[@id="js--hp-gallery-scorecard"]').attrib['data-review-score']
    except KeyError:
      score = None

    highlights = []

    highlights_path = response.xpath('/html/body/div[4]/div/div[5]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]/div/div/ul/li')

    for highlight in highlights_path:
      highlights.append(highlight.xpath('div/div/div/span/div/span/text()').get())

    yield {
      'city': city,
      'name': response.xpath('//h2[@class="d2fee87262 pp-header__title"]/text()').get(),
      'url': url,
      'score': score,
      'description': response.xpath('/html/body/div[4]/div/div[5]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[1]/div/div/p[1]/text()').get(),
      'contact_details': response.xpath('//a[@id="map_trigger_header_pin"]').attrib['data-atlas-latlng'],
      'highlights': highlights
    }


filename = "list_hotels.json"

process = CrawlerProcess(settings={
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'data/' + filename: {"format": "json"},
    }
})

process.crawl(BookingSpider)
process.start()

2025-01-05 17:03:50 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2025-01-05 17:03:50 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.15 3 Sep 2024), cryptography 42.0.5, Platform Windows-11-10.0.22000-SP0
2025-01-05 17:03:50 [scrapy.addons] INFO: Enabled addons:
[]
2025-01-05 17:03:50 [scrapy.extensions.telnet] INFO: Telnet Password: 9f3e9e7535613075
2025-01-05 17:03:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-01-05 17:03:50 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win

ReactorNotRestartable: 

Store in data lake

In [25]:
aws_settings = open("data/aws_settings.txt", "r").readlines()

session = boto3.Session(
    aws_access_key_id= aws_settings[0].strip(),
    aws_secret_access_key= aws_settings[1]
)

s3 = session.resource("s3")

bucketname = 'bucket-kayak-nico'

bucket = s3.Bucket(bucketname)

# Check si le bucket existe, création sinon
if bucket.creation_date == None:
  bucket = s3.create_bucket(
      Bucket=bucketname,
      CreateBucketConfiguration={
          'LocationConstraint': 'eu-west-3',
      },)

In [17]:
# Import des fichiers
bucket.upload_file("src/list_hotels.json", "list_hotels.json")